In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.layers import Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Input, ZeroPadding2D, Activation
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Add, MaxPooling2D, Flatten, Dense, Softmax, Activation, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD
import keras
import numpy as np
import h5py

In [2]:
tf.__version__
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
def dataGenerator(start_index, stop_index):
    def generator():
        indexes = shuff[start_index : stop_index]
        for i in indexes:
            x = np.array(hdf5['Xs'][i])
            y = np.array(hdf5['Ys'][i])
            yield x, y
    return generator

In [5]:
hdf5 = h5py.File('./hdf5/kyu_dataset.hdf5','r')
# for j in range (0, 13):
#     print(hdf5['xs'][50][j])
# xs = hdf5.get('Xs')
# ys = hdf5.get('Ys')
batch_size = 64
dataset_size = hdf5['Xs'].shape[0]
val_start = 0.8
val_stop = 1.0

train_steps = int(dataset_size * (val_start)/batch_size)
val_steps = int(dataset_size * (val_stop - val_start)/batch_size)
print(f"train_steps: {train_steps}, val_steps: {val_steps}")
shuff = np.arange(dataset_size)
np.random.shuffle(shuff)

data_gen = dataGenerator(start_index = 0, stop_index = int(dataset_size * val_start)) # 90% of the complete dataset
dataset = tf.data.Dataset.from_generator(data_gen, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,19)),tf.TensorShape((1))))
dataset = dataset.batch(batch_size, drop_remainder=True)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

data_gen_valid = dataGenerator(start_index=int(dataset_size * val_start), # 10% of the complete dataset
                               stop_index=int(dataset_size * val_stop))
dataset_valid = tf.data.Dataset.from_generator(data_gen_valid, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,19)),tf.TensorShape((1))))
dataset_valid = dataset_valid.batch(batch_size, drop_remainder=True)
dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)


train_steps: 169592, val_steps: 42398


In [6]:
# resnet34
# def residual_block(x, filters, kernel_size):
#     y = Conv2D(kernel_size=kernel_size,
#                filters=filters,
#                padding='same')(x)
#     y = ReLU()(y)
#     y = Conv2D(kernel_size=kernel_size,
#                filters=filters,
#                padding='same')(y)
#     output = Add()([x,y])
#     output = ReLU()(output)
#     return output

# def residual_block_s(x, filters, kernel_size):
#     y = Conv2D(kernel_size=kernel_size,
#                filters=filters,
#                padding='same')(x)
#     y = ReLU()(y)
#     y = Conv2D(kernel_size=kernel_size,
#                filters=filters,
#                padding='same')(y)
#     x = Conv2D(kernel_size = 1,
#               filters=filters)(x)
#     output = Add()([x,y])
#     output = ReLU()(output)
#     return output

# def go_res():
#     inputs = Input(shape=(19, 19, 19))
#     conv5x5 = Conv2D(kernel_size=5,
#                      filters=64,
#                      padding="same",
#                      name='conv5x5')(inputs)
#     conv1x1 = Conv2D(kernel_size=1,
#                      filters=64,
#                      padding="same",
#                      name='conv1x1')(inputs)
#     outputs = Add()([conv5x5, conv1x1])
#     outputs = ReLU()(outputs)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block_s(x=outputs,#
#                              filters=128,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=128,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=128,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=128,
#                              kernel_size=3)
#     outputs = residual_block_s(x=outputs,
#                              filters=256,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=256,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=256,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=256,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=256,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=256,
#                              kernel_size=3)
#     outputs = residual_block_s(x=outputs,#
#                              filters=512,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=512,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=512,
#                              kernel_size=3)
# #     outputs = Conv2D(kernel_size=3,
# #                      filters=1,
# #                      padding="same")(outputs)
# #     outputs = ReLU()(outputs)
# #     outputs = Flatten()(outputs)
# #     outputs = Softmax()(outputs)
#     outputs = GlobalAveragePooling2D()(outputs)
#     outputs = Dense(361, activation='softmax')(outputs)
#     model = Model(inputs, outputs)
    
#     opt = Adam(learning_rate=0.0001)
#     model.compile(optimizer=opt,
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])
#     return model


# model = go_res()
# model.summary()

In [7]:
# resnet50
def identity_block(input_tensor, kernel_size, filters):
    filters1, filters2, filters3 = filters
    bn_axis = 3

    # 這邊就是圖5上的1x1x64降維操作，假設input x的維度是(n, n, 256), channel last
    x = Conv2D(filters1, (1, 1))(input_tensor)
#     x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)
    
    # 正常的3x3x64卷積操作，Feature Map長寬仍是n x n
    x = Conv2D(filters2, kernel_size, padding='same')(x)
#     x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)
    
    # 最後升維到256，維度(n,n,256) -> 變成可以和(Indentity)input x相加的維度
    x = Conv2D(filters3, (1, 1))(x)
#     x = BatchNormalization(axis=bn_axis)(x)
    
    # 相加後做non-linear轉換
    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x
def conv_block(input_tensor,
               kernel_size,
               filters,
               strides=(1, 1)):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    
    # 因為是projection shortcut 所以input的x可能跟output維度不同
    # input維度(n,n,256) -->降維 (n,n,64)
    # 如果Strides有改，則利用Strides來改變Feature Map長寬
    x = Conv2D(filters1, (1, 1), strides=strides)(input_tensor)
#     x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)
    
    # (3,3)的kernel, padding都pad好pad滿，不改變Feature Map尺寸大小
    x = Conv2D(filters2, kernel_size, padding='same')(x)
#     x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)
    
    # 用1x1 conv升維到假設512
    x = Conv2D(filters3, (1, 1))(x)
#     x = BatchNormalization(axis=bn_axis)(x)
    
    # 因input維度是256，這邊就需要做projectr將維度升到512相加
    shortcut = Conv2D(filters3, (1, 1), strides=strides)(input_tensor)
#     shortcut = BatchNormalization(axis=bn_axis)(shortcut)
    # F(x) + x(升維後的x)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x
def ResNet50(include_top=True,
             input_tensor=None,
             input_shape=None,
             pooling=False):
    inputs = Input(shape = input_shape)
    
#     conv5x5 = Conv2D(kernel_size=5,
#                      filters=256,
#                      padding="same",
#                      name='conv5x5')(inputs)
#     conv1x1 = Conv2D(kernel_size=1,
#                      filters=256,
#                      padding="same",
#                      name='conv1x1')(inputs)
#     outputs = Add()([conv5x5, conv1x1])
#     outputs = ReLU()(outputs)
    x = conv_block(inputs, 3, [256, 256, 1024])  # input Channel大小會跟最後最後residual output尺寸一樣
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    # 256-d to 512-d
#     x = conv_block(x, 3, [128, 128, 512]) # projection shortcut
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    # 512-d to 1024-d
#     x = conv_block(x, 3, [256, 256, 1024]) # projection shortcut
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    # 1024-d to 2048-d
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
#     add
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    x = identity_block(x, 3, [256, 256, 1024]) 
    
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(361, activation='softmax', name='fc1000')(x)

    model = Model(inputs, x, name='resnet50')
    return model

model = ResNet50(include_top=True,
                 input_tensor=None,
                 input_shape=(19, 19, 19),
                 pooling=False)
opt = Nadam(learning_rate = 0.0001)
model.compile(optimizer = opt,
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 19, 19, 19)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 19, 19, 256)  5120        ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 19, 19, 256)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 19, 19, 256)  590080      ['activation[0][0]']      

 activation_13 (Activation)     (None, 19, 19, 256)  0           ['conv2d_14[0][0]']              
                                                                                                  
 conv2d_15 (Conv2D)             (None, 19, 19, 1024  263168      ['activation_13[0][0]']          
                                )                                                                 
                                                                                                  
 add_4 (Add)                    (None, 19, 19, 1024  0           ['conv2d_15[0][0]',              
                                )                                 'activation_11[0][0]']          
                                                                                                  
 activation_14 (Activation)     (None, 19, 19, 1024  0           ['add_4[0][0]']                  
                                )                                                                 
          

 conv2d_29 (Conv2D)             (None, 19, 19, 256)  590080      ['activation_27[0][0]']          
                                                                                                  
 activation_28 (Activation)     (None, 19, 19, 256)  0           ['conv2d_29[0][0]']              
                                                                                                  
 conv2d_30 (Conv2D)             (None, 19, 19, 1024  263168      ['activation_28[0][0]']          
                                )                                                                 
                                                                                                  
 add_9 (Add)                    (None, 19, 19, 1024  0           ['conv2d_30[0][0]',              
                                )                                 'activation_26[0][0]']          
                                                                                                  
 activatio

 activation_42 (Activation)     (None, 19, 19, 256)  0           ['conv2d_43[0][0]']              
                                                                                                  
 conv2d_44 (Conv2D)             (None, 19, 19, 256)  590080      ['activation_42[0][0]']          
                                                                                                  
 activation_43 (Activation)     (None, 19, 19, 256)  0           ['conv2d_44[0][0]']              
                                                                                                  
 conv2d_45 (Conv2D)             (None, 19, 19, 1024  263168      ['activation_43[0][0]']          
                                )                                                                 
                                                                                                  
 add_14 (Add)                   (None, 19, 19, 1024  0           ['conv2d_45[0][0]',              
          

 conv2d_58 (Conv2D)             (None, 19, 19, 256)  262400      ['activation_56[0][0]']          
                                                                                                  
 activation_57 (Activation)     (None, 19, 19, 256)  0           ['conv2d_58[0][0]']              
                                                                                                  
 conv2d_59 (Conv2D)             (None, 19, 19, 256)  590080      ['activation_57[0][0]']          
                                                                                                  
 activation_58 (Activation)     (None, 19, 19, 256)  0           ['conv2d_59[0][0]']              
                                                                                                  
 conv2d_60 (Conv2D)             (None, 19, 19, 1024  263168      ['activation_58[0][0]']          
                                )                                                                 
          

                                )                                                                 
                                                                                                  
 conv2d_73 (Conv2D)             (None, 19, 19, 256)  262400      ['activation_71[0][0]']          
                                                                                                  
 activation_72 (Activation)     (None, 19, 19, 256)  0           ['conv2d_73[0][0]']              
                                                                                                  
 conv2d_74 (Conv2D)             (None, 19, 19, 256)  590080      ['activation_72[0][0]']          
                                                                                                  
 activation_73 (Activation)     (None, 19, 19, 256)  0           ['conv2d_74[0][0]']              
                                                                                                  
 conv2d_75

                                                                                                  
 activation_86 (Activation)     (None, 19, 19, 1024  0           ['add_28[0][0]']                 
                                )                                                                 
                                                                                                  
 avg_pool (GlobalAveragePooling  (None, 1024)        0           ['activation_86[0][0]']          
 2D)                                                                                              
                                                                                                  
 fc1000 (Dense)                 (None, 361)          370025      ['avg_pool[0][0]']               
                                                                                                  
Total params: 32,487,017
Trainable params: 32,487,017
Non-trainable params: 0
_______________________________

In [8]:
callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                patience=3, verbose=0, mode='min'
                                 , restore_best_weights=True)
# callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
#                                 patience=3, verbose=0, mode='max'
#                                  , restore_best_weights=True)
callback2 = keras.callbacks.ModelCheckpoint('./models/model_kyu_b64_f256_l88_{epoch:02d}_{val_accuracy:.4f}.h5', 
                                            monitor='val_accuracy', 
                                            verbose=0, save_best_only=False, save_weights_only=False, 
                                            mode='max', save_freq="epoch")

In [9]:
# model = load_model('./models/model_kyu_f256_l88_mir_02_0.0035.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 500,
#     steps_per_epoch = train_steps,
    validation_data = dataset_valid,
#     validation_steps = val_steps,
    callbacks = [callback1, callback2]
)

Epoch 1/500
169592/169592 [==============================] - 60183s 355ms/step - loss: 2.1976 - accuracy: 0.4784 - val_loss: 1.8881 - val_accuracy: 0.5102
Epoch 2/500
169592/169592 [==============================] - 60941s 359ms/step - loss: 1.8092 - accuracy: 0.5231 - val_loss: 1.7836 - val_accuracy: 0.5242
Epoch 3/500
169592/169592 [==============================] - 60651s 358ms/step - loss: 1.7341 - accuracy: 0.5342 - val_loss: 1.7484 - val_accuracy: 0.5291
Epoch 4/500
169592/169592 [==============================] - 63740s 376ms/step - loss: 1.6944 - accuracy: 0.5402 - val_loss: 1.7220 - val_accuracy: 0.5346
Epoch 5/500
 26491/169592 [===>..........................] - ETA: 14:44:25 - loss: 1.6786 - accuracy: 0.5427

In [8]:
result = model.evaluate(
    dataset_valid,
    steps = val_steps
)

11374/21176 [===============>..............] - ETA: 43:34 - loss: 1.6519 - accuracy: 0.5500

KeyboardInterrupt: 

model.save('./models/model_kyu_resnet50_withpp_1.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!